# BP for MLP

## 1. Forward

设共有$L$层

$ z^l \in\mathbb R^{n}$ 为第 $ l $ 层经过激活函数前的输出<br>
$ a^l \in\mathbb R^{m} $ 为第 $ l $ 层经过激活函数后的输出<br>
$ \sigma $ 为激活函数<br>
$ \mathbf W^l \in\mathbb R^{n\times m} $ 为第 $ l $ 层的权重矩阵<br>
$ b^l \in\mathbb R^{n} $ 为第 $ l $ 层的偏置项<br>

$$ z^l = \mathbf W^la^{l-1}+b^l $$

$$ a^l = \sigma(z^l) $$

## 2.Backward

最终误差:$ C $

第$l$层的误差: $\delta^l$, 满足$$\delta^l=\frac{\partial C}{\partial z^l}$$

例如，对于MSE，$$C=\frac 1 2 \left\|a^L-y\right\|^2_2$$

输出层的误差$$\delta^L=\frac{\partial C}{\partial z^L}=\frac{\partial C}{\partial a^L}\frac{\partial a^L}{\partial z^L}=(a^L-y)\odot\sigma'(z^L)$$
其中，$\odot$为Hadmard积

接着求最终误差$C$对于输出层权重矩阵和偏置的偏导

$$\frac{\partial C}{\partial \mathbf W^L}=\frac{\partial C}{\partial z^L}\frac{\partial z^L}{\partial \mathbf W^L}=\delta^L(a^{L-1})^T$$

ps: 矩阵求导时，即乘上系数的转置，左乘还是右乘和求导前一致

$$\frac{\partial C}{\partial b^L}=\frac{\partial C}{\partial z^L}\frac{\partial z^L}{\partial b^L}=\delta^L\odot\mathbf 1=\delta^L$$

接着求第$l$层的误差$\delta^l$
$$ \delta^l=\frac{\partial C}{\partial z^l}
           =\frac{\partial C}{\partial z^{l+1}}\frac{\partial z^{l+1}}{\partial z^{l}}
           =\delta^{l+1}\frac{\partial z^{l+1}}{\partial z^{l}}
$$

又：
$$
z^{l+1}=\mathbf W^{l+1}a^l+b^{l+1}
       =\mathbf W^{l+1}\sigma(z^l)+b^{l+1}
$$

求得：
$$
z^{l+1}
=(\mathbf W^{l+1})^T\delta^{l+1}\odot\sigma'(z^l)
$$